In [ ]:
!wget http://setup.johnsnowlabs.com/kaggle.sh -O - | bash 

In [ ]:
pip install neo4j

In [ ]:
from neo4j import __version__ as neo4j_version
print(neo4j_version)

In [ ]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [ ]:
conn = Neo4jConnection(uri="neo4j+s://1450681e.databases.neo4j.io", user="neo4j", pwd="a9igi8HfgRgXv8VRRr2U91RsTol6cjS2ZO7ZUj6QtPc")

In [ ]:
from neo4j import GraphDatabase

#establish connection
graphdp = GraphDatabase.driver(uri="neo4j+s://1450681e.databases.neo4j.io", auth=("neo4j","a9igi8HfgRgXv8VRRr2U91RsTol6cjS2ZO7ZUj6QtPc"))

session = graphdp.session()


In [ ]:

query = ''' MATCH (t:text)-[g:`has written`]-(r:review_id)-[h:`has stars`]-(s:stars) RETURN t,s ''' 
result = session.run(query).data() 
df = pd.DataFrame(result)
a2 = spark.createDataFrame(df)
review=a2.select('t.text','s.stars')
review.show()

In [ ]:
review=a2.select('t.text','s.stars')
review.show()

In [ ]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer

#!pip install -q findspark
import pyspark.sql.functions as F
#import findspark
import os
import sys
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import math
import folium
from IPython.core.interactiveshell import InteractiveShell
from datetime import *
import statistics as stats
# This helps auto print out the items without explixitly using 'print'
InteractiveShell.ast_node_interactivity = "all" 
%matplotlib inline
# Import PySpark related modules
import pyspark
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions
from pyspark.sql.functions import lit, desc, col, size, array_contains\
, isnan, udf, hour, array_min, array_max, countDistinct
from pyspark.sql.types import *

Data Labeling

In [ ]:
from  pyspark.sql.functions import * 
import pyspark.sql.functions as F
from pyspark.sql import functions
def for_exist_column(df, col, pre):
    if col in df.columns:
        return pre(df[col])
    else:
        return F.lit(False)
review_neutral=review.select('*').where(review['stars']==3)
review=review.join(review_neutral,how='leftanti',on='text')
review_neutral=review_neutral.withColumn('Target_sentiment',F.when(for_exist_column(review_neutral, 'stars', lambda c: c==3), 'neutral'))
review=review.withColumn("Target_sentiment",F.when(for_exist_column(review, 'stars', lambda c: c>3), 'positive').otherwise('negative'))
review=review.withColumn( 'text',regexp_replace('text',r',|\.|&|\\|\||-|_|!|.', ''))
review_positive= review.select('text','stars','Target_sentiment').where(review['Target_sentiment'] =='positive')
review_positive=review_positive.orderBy(rand()).limit(5000)
review_negative= review.select('text','stars','Target_sentiment').where(review['Target_sentiment'] =='negative')
review_negative=review_negative.orderBy(rand()).limit(5000)
review_neutral=review_neutral.orderBy(rand()).limit(5000)
reviewf=review_positive.union(review_negative)
reviewf=reviewf.union(review_neutral)

La lemmatizzazione è il processo di riduzione di una forma flessa di una parola alla sua forma canonica (non marcata), detta lemma.

In [ ]:
from collections import Counter
import itertools as it
import seaborn as sns
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
document= DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")
sentence=SentenceDetector()\
.setInputCols(['document'])\
.setOutputCol('sentence')
token=Tokenizer()\
.setInputCols(['sentence'])\
.setOutputCol('token')
stop_words=StopWordsCleaner.pretrained('stopwords_en','en')\
.setInputCols(['token'])\
.setOutputCol('cleanTokens')\
.setCaseSensitive(False)
lemmatizer=LemmatizerModel.pretrained()\
.setInputCols(['cleanTokens'])\
.setOutputCol('lemma')
finisher=Finisher()\
.setInputCols(['lemma'])\
.setOutputCols(['token_features'])\
.setOutputAsArray(True)\
.setCleanAnnotations(False)

In [ ]:

def plot_histogram(data,x,y,t,u):
    ## Plot Distribution of the reviews
    data=data.select(u).toPandas().to_numpy().ravel()
    
    plt.style.use('ggplot')
    fig, ax = plt.subplots(figsize=(7,5))
    plt.hist(data, bins=50, color = "tab:red")
    ax.set_title(t)
    ax.set_xlabel(x)
    ax.set_ylabel(y)




    
a1=review_neutral.rdd.map(lambda x: [x['text'],len(x['text'])])
schema = StructType([StructField('text', StringType(), True),StructField('Text_Length', IntegerType(), True)])
a1 = spark.createDataFrame(a1,schema)
a2=review_neutral.rdd.map(lambda x: [x['text'],len(x['text'].split())])
schema = StructType([StructField('text', StringType(), True),StructField('Word_Count', IntegerType(), True)])
a2 = spark.createDataFrame(a2,schema)

a1=a1.join(a2,on='text',how='left')
nlp_pipeline=Pipeline(
stages=[document,
sentence,
token,
stop_words,
lemmatizer,
finisher])
model=nlp_pipeline.fit(a1)
a=model.transform(a1)

plot_histogram(a, x = "Text length", y = "count of reviews", t = 'Text length plot of the reviews',u='Text_Length')
plot_histogram(a, x = "Word count", y = "count of reviews", t = 'Word Count plot of the reviews',u='Word_Count')
    


In [ ]:
h=a.select('lemma.result').toPandas().to_numpy().ravel()
k=[]
for i in range (len(h)):
    k.append(h[i])
k=list(map(str, it.chain.from_iterable(k)))



In [ ]:
j=Counter(k)
j.pop('.')
j.pop(',')
j.pop('!')
j.pop('(')
j.pop(')')
j.pop('&')
j.pop('!!')
j.pop('-')
j.pop('=')
j.pop('"')
j.most_common()

In [ ]:
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

In [ ]:
train_set, test_set = reviewf.randomSplit([0.8,0.2],seed=100)

In [ ]:

print('Train')
train_set.groupBy('Target_sentiment').count().show()
print('Test')
test_set.groupBy('Target_sentiment').count().show()
#print(test_set.count())
print('General')
reviewf.groupBy('Target_sentiment').count().show()
#print(train_set.count())


In [ ]:
document= DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")
sentence=SentenceDetector()\
.setInputCols(['document'])\
.setOutputCol('sentence')
token=Tokenizer()\
.setInputCols(['sentence'])\
.setOutputCol('token')
stop_words=StopWordsCleaner.pretrained('stopwords_en','en')\
.setInputCols(['token'])\
.setOutputCol('cleanTokens')\
.setCaseSensitive(False)
lemmatizer=LemmatizerModel.pretrained()\
.setInputCols(['cleanTokens'])\
.setOutputCol('lemma')
finisher=Finisher()\
.setInputCols(['lemma'])\
.setOutputCols(['token_features'])\
.setOutputAsArray(True)\
.setCleanAnnotations(False)

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, StringIndexer, IndexToString
from pyspark.ml.classification import LogisticRegression, NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [ ]:
hashTF=HashingTF(inputCol="token_features",outputCol="raw_features")
idf=IDF(inputCol="raw_features",outputCol="features",minDocFreq=5)
label_strIdx=StringIndexer(inputCol='Target_sentiment',outputCol="label")
LogReg=LogisticRegression(maxIter=10)
label_idxStr=IndexToString(inputCol="label",outputCol="class")
nlp_pipeline=Pipeline(
stages=[document,
       sentence,
       token,
       stop_words,
       lemmatizer,
       finisher,
       hashTF,
       idf,
        label_strIdx,
       LogReg,
       label_idxStr])
clas_model=nlp_pipeline.fit(train_set)

In [ ]:
pred=clas_model.transform(test_set)
pred.write.json('pre.json')

In [ ]:
evaluator=MulticlassClassificationEvaluator(
labelCol='label',predictionCol='prediction',metricName='accuracy')
accuracy=evaluator.evaluate(pred)
print('Accuracy=%g' %(accuracy))
print('Test Error =%g' %(1.0-accuracy))

In [ ]:
from sklearn.metrics import classification_report,accuracy_score
df_lr=clas_model\
.transform(test_set)\
.select('Target_sentiment','label','prediction')\
.toPandas()
df_lr.head()
print(classification_report(df_lr.label,df_lr.prediction))

In [ ]:
import pandas as pd
from pyspark.mllib.evaluation import MulticlassMetrics
predictionAndLabels = pred.select('label', 'prediction')
metrics = MulticlassMetrics(predictionAndLabels.rdd.map(lambda x: tuple(map(float, x))))

confusion_matrix = metrics.confusionMatrix().toArray()
labels = [int(l) for l in metrics.call('labels')]
confusion_matrix = pd.DataFrame(confusion_matrix , index=labels, columns=labels)

In [ ]:
confusion_matrix

2 approccio: Naive Bayes

In [ ]:
hashTF=HashingTF(inputCol="token_features",outputCol="raw_features")
idf=IDF(inputCol="raw_features",outputCol="features",minDocFreq=5)
label_strIdx=StringIndexer(inputCol='Target_sentiment',outputCol="label")
bayes_class=NaiveBayes(smoothing=111)
label_idxStr=IndexToString(inputCol="label",outputCol="class")
nlp_pipeline_b=Pipeline(
stages=[document,
       sentence,
       token,
       stop_words,
       lemmatizer,
       finisher,
       hashTF,
       idf,
        label_strIdx,
       bayes_class,
       label_idxStr])
clas_model_naive=nlp_pipeline_b.fit(train_set)

In [ ]:
pred_2=clas_model_naive.transform(test_set)
evaluator=MulticlassClassificationEvaluator(
labelCol='label',predictionCol='prediction',metricName='accuracy')
accuracy_2=evaluator.evaluate(pred_2)
print('Accuracy=%g' %(accuracy_2))
print('Test Error =%g' %(1.0-accuracy_2))
from sklearn.metrics import classification_report,accuracy_score
df_lr_2=clas_model_naive\
.transform(test_set)\
.select('Target_sentiment','label','prediction')\
.toPandas()



In [ ]:
df_lr_2.head()
print(classification_report(df_lr_2.label,df_lr_2.prediction))
import pandas as pd
from pyspark.mllib.evaluation import MulticlassMetrics
predictionAndLabels_2 = pred_2.select('label', 'prediction')
metrics_2 = MulticlassMetrics(predictionAndLabels_2.rdd.map(lambda x: tuple(map(float, x))))

confusion_matrix2 = metrics_2.confusionMatrix().toArray()
labels_2 = [int(l) for l in metrics_2.call('labels')]
confusion_matrix2 = pd.DataFrame(confusion_matrix2 , index=labels_2, columns=labels_2)
confusion_matrix2

UNIVERSAL SENTENCE DETECTOR

In [ ]:
from sparknlp.pretrained import PretrainedPipeline

In [ ]:

document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    

use = UniversalSentenceEncoder.pretrained()\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

classsifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("Target_sentiment")\
  .setMaxEpochs(10)\
  .setEnableOutputLogs(True)
use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

In [ ]:
use_pipelineModel = use_clf_pipeline.fit(train_set)

In [ ]:
preds=use_pipelineModel.transform(test_set)
from sklearn.metrics import classification_report, accuracy_score
df=preds.select('text','Target_sentiment','class.result').toPandas()
df['result']=df['result'].apply(lambda x: x[0])
print(classification_report(df.Target_sentiment,df.result))
print(accuracy_score(df.Target_sentiment,df.result))

-----

BERT

In [ ]:
from collections import Counter
import itertools as it
import seaborn as sns
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
document = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

bert_sent = BertSentenceEmbeddings.pretrained('sent_small_bert_L8_512')\
      .setInputCols(["document"])\
      .setOutputCol("sentence_embeddings")


classsifierdl = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("Target_sentiment")\
      .setMaxEpochs(10)\
      .setEnableOutputLogs(True)\
      .setLr(0.001)

bert_sent_clf_pipeline = Pipeline(
    stages = [
        document,
        bert_sent,
        classsifierdl
    ])

In [ ]:
bert_sent_pipelineModel = bert_sent_clf_pipeline.fit(train_set)

In [ ]:
from sklearn.metrics import classification_report

preds = bert_sent_pipelineModel.transform(test_set)

preds_df = preds.select('Target_sentiment','text',"class.result").toPandas()

preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

print (classification_report(preds_df['result'], preds_df['Target_sentiment']))

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics
predictionAndLabels_2 = preds.select('label', 'prediction')
metrics_2 = MulticlassMetrics(predictionAndLabels_2.rdd.map(lambda x: tuple(map(float, x))))

confusion_matrix2 = metrics_2.confusionMatrix().toArray()
labels_2 = [int(l) for l in metrics_2.call('labels')]
confusion_matrix2 = pd.DataFrame(confusion_matrix2 , index=labels_2, columns=labels_2)
confusion_matrix2

GRAFICI

In [ ]:
h=[0.88,0.69,0.78,0.76,0.93]
lab=['LogReg','Naive','RandomF','U.S.E','BERT']
import matplotlib.pyplot as plt

plt.bar(lab,h)
plt.xlabel('Approach')
plt.ylabel('Accuracy')

In [ ]:
laab=['negative','neutral','positive']
errorilog=[129,111,105]
errNaiv=[675,2,249]
errRand=[6,491,165]
errBert=[110,19,61]
err=[(129+675+6+110)/4,(111+2+491+19)/4,(105+249+165+61)/4]
plt.bar(laab,err)

-----

Random Forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
hashTF=HashingTF(inputCol="token_features",outputCol="raw_features")
idf=IDF(inputCol="raw_features",outputCol="features",minDocFreq=5)
label_strIdx=StringIndexer(inputCol='Target_sentiment',outputCol="label")
label_idxStr=IndexToString(inputCol="label",outputCol="class")
nlp_pipeline_b=Pipeline(
stages=[document,
       sentence,
       token,
       stop_words,
       lemmatizer,
       finisher,
       hashTF,
       idf,
        label_strIdx,
       label_idxStr])
clas_modell=nlp_pipeline_b.fit(train_set)
utile_test=clas_modell.transform(test_set)


In [ ]:
utile_train=clas_modell.transform(train_set)
algo = RandomForestClassifier(featuresCol='features', labelCol='label')

model = algo.fit(utile_train)
predictions = model.transform(utile_test)

In [ ]:
evaluator=MulticlassClassificationEvaluator(
labelCol='label',predictionCol='prediction',metricName='accuracy')
accuracy_2=evaluator.evaluate(predictions)
print('Accuracy=%g' %(accuracy_2))
print('Test Error =%g' %(1.0-accuracy_2))
from sklearn.metrics import classification_report,accuracy_score
df_lr_2=clas_modell\
.transform(test_set)\
.select('Target_sentiment','label','prediction')\
.toPandas()

In [ ]:
from sklearn.metrics import classification_report,accuracy_score
df_lr=model\
.transform(utile_test)\
.select('Target_sentiment','label','prediction')\
.toPandas()
df_lr.head()
print(classification_report(df_lr.label,df_lr.prediction))
import pandas as pd
from pyspark.mllib.evaluation import MulticlassMetrics
predictionAndLabels = predictions.select('label', 'prediction')
metrics = MulticlassMetrics(predictionAndLabels.rdd.map(lambda x: tuple(map(float, x))))

confusion_matrix = metrics.confusionMatrix().toArray()
labels = [int(l) for l in metrics.call('labels')]
confusion_matrix = pd.DataFrame(confusion_matrix , index=labels, columns=labels)

In [ ]:
confusion_matrix

SALVATAGGIO SU MONGO

In [ ]:
pip install pymongo

In [ ]:
#mongodb+srv://luigigarofalo:<password>@cluster0.sy8pl.mongodb.net/?retryWrites=true&w=majority
# Imports
#!pip install dnspython
#!pip install pymongo
import pandas as pd
from pymongo import MongoClient
# Load csv dataset
data =pred.select('text','label','prediction').toPandas()
# Connect to MongoDB
client =  MongoClient("mongodb+srv://luigigarofalo:2SenxCosx<@cluster0.sy8pl.mongodb.net/?retryWrites=true&w=majority")
db = client['results']
collection = db['logreg']
data.reset_index(inplace=True)
data_dict = data.to_dict("records")
# Insert collection
collection.insert_many(data_dict)